In [1]:
%%capture
import sys
!{sys.executable} -m pip install requests pandas geopy bs4 folium sklearn matplotlib

In [2]:
import folium
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import json
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
# Setting credentials required by foursquare
CLIENT_ID = 'DEPKYDHCHC40DUDYBECM3Y1GYBGOFGRTODGR4QLNQ5VPNLBU'
CLIENT_SECRET = 'FWKDHU4URB1WHQI5IKQ5QN4RMLEWOASA34AHLW3YNA4KCGH5'
VERSION = '20190808'

<h4>Scrape list of areas in Pune city from "makemytrip" website</h4>

In [4]:
# Function to scrape data from website and convert it into dataframe
geolocator = Nominatim(user_agent="coursera agent")
def create_areas_data_frame(): 
    with open('area_index.html') as f:
        soup = BeautifulSoup(f, parser='lxml')

    city_areas = soup.find_all('div', attrs={'class': 'city-listing'})
    areas_dict = {'Area': [], 'Latitude': [], 'Longitude': []}
    for areas in city_areas:
        area_cat = areas.find('p', attrs={'class': 'key-values'})
        for area_parsed in area_cat.find_all('a'):
            area_short = area_parsed.text.split(',')[0]
            area = '{}, Pune'.format(area_short)
            try:
                location = geolocator.geocode(area)
                latitude = location.latitude
                longitude = location.longitude
            except Exception as e:
                pass
            else:
                areas_dict['Area'].append(area_short)
                areas_dict['Latitude'].append(latitude)
                areas_dict['Longitude'].append(longitude)

    df_areas = pd.DataFrame(areas_dict)
    # df_areas.to_csv('pune_areas.csv')
    return df_areas

Load data in a dataframe and discard areas which lie in the outskirts of the city

In [5]:
# Load scraped data from csv
df_pune = pd.read_csv('pune_areas.csv')
df_pune.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df_pune = df_pune[(df_pune['Latitude'] < 18.6) & (df_pune['Latitude'] > 18.35)
                  & (df_pune['Longitude'] < 74.0) & (df_pune['Longitude'] > 73.6)]
print(df_pune.shape)
df_pune.head(10)

(69, 3)


,Area,Latitude,Longitude
0,Aundh,18.561883,73.810196
3,Anand Nagar,18.478490,73.821326
5,Balewadi,18.582027,73.768983
6,Baner,18.564243,73.776857
7,Bund Garden Road,18.533404,73.878625
10,Budhwar Peth,18.517560,73.858037
11,Baner,18.564243,73.776857
12,Central Pune,18.520306,73.854319
17,Dhole Patil Road,18.534908,73.876258
18,Deccan Gymkhana,18.515893,73.841160


In total, there are 69 areas in Pune. The cell above shows dataframe with areas and their latitudes and longitudes

In [6]:
location = geolocator.geocode('Pune')
pune_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)
for lat, lon, area in zip(df_pune['Latitude'], df_pune['Longitude'], df_pune['Area']):
    label = folium.Popup(area)
    folium.CircleMarker(
        [lat, lon],
        color='blue',
        popup=label,
        radius=8,
        fill_color='yellow',
        fill=True,
        fill_opacity=0.8
    ).add_to(pune_map)
pune_map

In [7]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=restaurant'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4>Get list of restaurants under each area</h4>

In [8]:
pune_venues = getNearbyVenues(names=df_pune['Area'],
                                   latitudes=df_pune['Latitude'],
                                   longitudes=df_pune['Longitude']
                                  )

Aundh
Anand Nagar
Balewadi
Baner
Bund Garden Road
Budhwar Peth
Baner
Central Pune
Dhole Patil Road
Deccan Gymkhana
Dapodi
Fergusson College Road
Fursungi
Fatima Nagar
Ghorpadi
Gultekdi
Hadapsar
Hinjewadi
Hinjewadi
Kothrud
Kalyani Nagar
Karve Nagar
Koregaon Park
Kharadi
Katraj
Kondhwa
Khadki
Laxmi Road
Magarpatta
Mundhwa
Mangalwar Peth
MG Road
Mumbai - Bangalore Highway
Mundhwa - Kharadi Road
Municipal Colony
Nagar Road
Nagar - Pune Highway
New Airport Road
Nana Peth
Narayan Peth
Narhe
NIA Campus Road
Pirangut
Pune - Solapur Highway
Pimple Saudagar
Pune Central City
Pune
Ragvilas Society
Revenue Colony
Senapati Bapat Road
Shaniwar Peth
Somwar Peth
Sadashiv Peth
Sinhagad Road
Swargate
Sangamwadi Road
Shukrawar Peth
Sky Lounge
Saswad Road
Swargate
University Road
Viman Nagar
Vitthalrao Navale Rd
Vishrantwadi
Wadgaon Sheri
Wanowrie
Ward No. 8
Wagholi
Yerwada


In [9]:
print(pune_venues.shape)
pune_venues.head()

(623, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aundh,18.561883,73.810196,Picantos Mexican Grill,18.560654,73.812447,Mexican Restaurant
1,Aundh,18.561883,73.810196,Venkys Xprs,18.560550,73.808964,Fast Food Restaurant
2,Aundh,18.561883,73.810196,Café Maroo,18.564801,73.809141,Korean Restaurant
3,Aundh,18.561883,73.810196,The Loft,18.562714,73.807630,Indian Restaurant
4,Aundh,18.561883,73.810196,Hyderabadi Spice,18.560594,73.810108,Indian Restaurant


API lists down 623 total restaurants in the city of Pune.

In [10]:
# Group by neighborhood/area
pune_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Anand Nagar,9,9,9,9,9,9
Aundh,22,22,22,22,22,22
Baner,28,28,28,28,28,28
Budhwar Peth,5,5,5,5,5,5
Bund Garden Road,27,27,27,27,27,27
...,...,...,...,...,...,...
Wadgaon Sheri,2,2,2,2,2,2
Wagholi,5,5,5,5,5,5
Wanowrie,13,13,13,13,13,13


In [11]:
print('There are {} uniques categories.'.format(len(pune_venues['Venue Category'].unique())))

There are 50 uniques categories.


In [12]:
# one hot encoding
pune_onehot = pd.get_dummies(pune_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pune_onehot['Neighborhood'] = pune_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [pune_onehot.columns[-1]] + list(pune_onehot.columns[:-1])
pune_onehot = pune_onehot[fixed_columns]

pune_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,...,Pizza Place,Punjabi Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South Indian Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
pune_onehot.shape

(623, 51)

In [14]:
pune_grouped = pune_onehot.groupby('Neighborhood').mean().reset_index()
pune_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,...,Pizza Place,Punjabi Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,South Indian Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Anand Nagar,0.0,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,...,0.111111,0.0,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.0,0.000000
1,Aundh,0.0,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.0,0.000000,...,0.000000,0.0,0.090909,0.000000,0.000000,0.045455,0.000000,0.0,0.0,0.045455
2,Baner,0.0,0.0,0.000000,0.000000,0.000000,0.071429,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.142857,0.142857,0.071429,0.0,0.0,0.000000
3,Budhwar Peth,0.0,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,Bund Garden Road,0.0,0.0,0.037037,0.074074,0.037037,0.037037,0.037037,0.0,0.000000,...,0.074074,0.0,0.037037,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Wadgaon Sheri,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.500000
57,Wagholi,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.200000,0.0,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000
58,Wanowrie,0.0,0.0,0.000000,0.153846,0.000000,0.076923,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.076923,0.000000,0.076923,0.000000,0.0,0.0,0.000000
59,Ward No. 8,0.0,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.166667,...,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.000000


In [15]:
pune_grouped.shape

(61, 51)

Listing down top five types of restaurants in each area

In [16]:
num_top_venues = 5

for hood in pune_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = pune_grouped[pune_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anand Nagar----
                  venue  freq
0                Bakery  0.33
1  Fast Food Restaurant  0.22
2     Indian Restaurant  0.11
3           Snack Place  0.11
4                  Café  0.11


----Aundh----
                  venue  freq
0     Indian Restaurant  0.27
1  Fast Food Restaurant  0.23
2            Restaurant  0.09
3        Breakfast Spot  0.09
4            Food Court  0.09


----Baner----
                     venue  freq
0        Indian Restaurant  0.29
1     Fast Food Restaurant  0.21
2              Snack Place  0.14
3       Seafood Restaurant  0.14
4  South Indian Restaurant  0.07


----Budhwar Peth----
                     venue  freq
0        Indian Restaurant   0.6
1                   Bakery   0.2
2        Indian Sweet Shop   0.2
3  North Indian Restaurant   0.0
4         Kebab Restaurant   0.0


----Bund Garden Road----
                  venue  freq
0     Indian Restaurant  0.33
1    Chinese Restaurant  0.11
2                  Café  0.07
3                Baker

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create area-wise top 10 most common type of restaurants/food outlets

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pune_grouped['Neighborhood']

for ind in np.arange(pune_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pune_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anand Nagar,Bakery,Fast Food Restaurant,Café,Indian Restaurant,Snack Place,Pizza Place,Comfort Food Restaurant,Food Court,Food,Falafel Restaurant
1,Aundh,Indian Restaurant,Fast Food Restaurant,Food Court,Breakfast Spot,Restaurant,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Snack Place
2,Baner,Indian Restaurant,Fast Food Restaurant,Snack Place,Seafood Restaurant,South Indian Restaurant,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court
3,Budhwar Peth,Indian Restaurant,Indian Sweet Shop,Bakery,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
4,Bund Garden Road,Indian Restaurant,Chinese Restaurant,Café,Pizza Place,Bakery,Fast Food Restaurant,Kebab Restaurant,Italian Restaurant,Buffet,Restaurant


In [ ]:
Use k-means clustering method to cluster city in 3 segments

In [19]:
# set number of clusters
kclusters = 3

pune_grouped_clustering = pune_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pune_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 4, 1, 2, 0, 1, 1, 1], dtype=int32)

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pune_merged = df_pune

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pune_merged = pune_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Area')

pune_merged.head() # check the last columns!

,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aundh,18.561883,73.810196,1.0,Indian Restaurant,Fast Food Restaurant,Food Court,Breakfast Spot,Restaurant,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Snack Place
3,Anand Nagar,18.478490,73.821326,0.0,Bakery,Fast Food Restaurant,Café,Indian Restaurant,Snack Place,Pizza Place,Comfort Food Restaurant,Food Court,Food,Falafel Restaurant
5,Balewadi,18.582027,73.768983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Baner,18.564243,73.776857,1.0,Indian Restaurant,Fast Food Restaurant,Snack Place,Seafood Restaurant,South Indian Restaurant,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court
7,Bund Garden Road,18.533404,73.878625,1.0,Indian Restaurant,Chinese Restaurant,Café,Pizza Place,Bakery,Fast Food Restaurant,Kebab Restaurant,Italian Restaurant,Buffet,Restaurant


In [21]:
pune_merged.dropna(axis=0, inplace=True)

In [22]:
pune_merged['Cluster Labels'] = pune_merged['Cluster Labels'].astype(int)
pune_merged.head(20)

,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aundh,18.561883,73.810196,1,Indian Restaurant,Fast Food Restaurant,Food Court,Breakfast Spot,Restaurant,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Snack Place
3,Anand Nagar,18.478490,73.821326,0,Bakery,Fast Food Restaurant,Café,Indian Restaurant,Snack Place,Pizza Place,Comfort Food Restaurant,Food Court,Food,Falafel Restaurant
6,Baner,18.564243,73.776857,1,Indian Restaurant,Fast Food Restaurant,Snack Place,Seafood Restaurant,South Indian Restaurant,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court
7,Bund Garden Road,18.533404,73.878625,1,Indian Restaurant,Chinese Restaurant,Café,Pizza Place,Bakery,Fast Food Restaurant,Kebab Restaurant,Italian Restaurant,Buffet,Restaurant
10,Budhwar Peth,18.517560,73.858037,4,Indian Restaurant,Indian Sweet Shop,Bakery,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
11,Baner,18.564243,73.776857,1,Indian Restaurant,Fast Food Restaurant,Snack Place,Seafood Restaurant,South Indian Restaurant,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court
12,Central Pune,18.520306,73.854319,2,Indian Restaurant,Snack Place,Bakery,Burrito Place,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant
17,Dhole Patil Road,18.534908,73.876258,1,Indian Restaurant,Chinese Restaurant,Bakery,Café,Italian Restaurant,Pizza Place,Fast Food Restaurant,Mediterranean Restaurant,Bistro,Restaurant
18,Deccan Gymkhana,18.515893,73.841160,1,Indian Restaurant,Café,Vegetarian / Vegan Restaurant,Snack Place,Fast Food Restaurant,Restaurant,Pizza Place,Bakery,Sandwich Place,Bistro
21,Dapodi,18.580846,73.832775,0,Food Court,Breakfast Spot,Restaurant,Cafeteria,Chinese Restaurant,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food,Fast Food Restaurant


In [29]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pune_merged['Latitude'], pune_merged['Longitude'], pune_merged['Area'], pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [24]:
# Cluster 0
pune_merged.loc[pune_merged['Cluster Labels'] == 0, pune_merged.columns[[1] + list(range(5, pune_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,18.478490,Fast Food Restaurant,Café,Indian Restaurant,Snack Place,Pizza Place,Comfort Food Restaurant,Food Court,Food,Falafel Restaurant
21,18.580846,Breakfast Spot,Restaurant,Cafeteria,Chinese Restaurant,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food,Fast Food Restaurant
35,18.537553,Asian Restaurant,Indian Restaurant,Middle Eastern Restaurant,Restaurant,Italian Restaurant,Fast Food Restaurant,Café,Creperie,Vegetarian / Vegan Restaurant
40,18.477746,Vegetarian / Vegan Restaurant,Indian Restaurant,Fast Food Restaurant,Pizza Place,Café,Asian Restaurant,Bakery,BBQ Joint,Breakfast Spot
56,18.557350,Thai Restaurant,BBQ Joint,Bakery,Indian Restaurant,Restaurant,Italian Restaurant,Pizza Place,Food Truck,Creperie
57,18.589521,Snack Place,Fast Food Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Food,Falafel Restaurant,Eastern European Restaurant
62,18.573425,Café,Fast Food Restaurant,Breakfast Spot,Comfort Food Restaurant,Food Truck,Food Court,Food,Falafel Restaurant,Eastern European Restaurant
79,18.525574,Restaurant,Snack Place,Sandwich Place,Pizza Place,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food,Fast Food Restaurant,Falafel Restaurant
82,18.521766,Café,Comfort Food Restaurant,Indian Restaurant,Seafood Restaurant,Sandwich Place,Fast Food Restaurant,Food Court,Food,Falafel Restaurant
102,18.518231,Café,Indian Restaurant,Fast Food Restaurant,Comfort Food Restaurant,Creperie,Gastropub,Food Truck,Food Court,Food


In [25]:
# Cluster 1
pune_merged.loc[pune_merged['Cluster Labels'] == 1, pune_merged.columns[[1] + list(range(5, pune_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,18.561883,Fast Food Restaurant,Food Court,Breakfast Spot,Restaurant,Korean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Snack Place
6,18.564243,Fast Food Restaurant,Snack Place,Seafood Restaurant,South Indian Restaurant,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court
7,18.533404,Chinese Restaurant,Café,Pizza Place,Bakery,Fast Food Restaurant,Kebab Restaurant,Italian Restaurant,Buffet,Restaurant
11,18.564243,Fast Food Restaurant,Snack Place,Seafood Restaurant,South Indian Restaurant,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court
17,18.534908,Chinese Restaurant,Bakery,Café,Italian Restaurant,Pizza Place,Fast Food Restaurant,Mediterranean Restaurant,Bistro,Restaurant
18,18.515893,Café,Vegetarian / Vegan Restaurant,Snack Place,Fast Food Restaurant,Restaurant,Pizza Place,Bakery,Sandwich Place,Bistro
23,18.524148,Burger Joint,Asian Restaurant,Sandwich Place,Restaurant,Donut Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Burrito Place
25,18.504510,Breakfast Spot,Pizza Place,Café,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant
28,18.493894,Burger Joint,Vegetarian / Vegan Restaurant,Creperie,Gastropub,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant
30,18.592094,Indian Restaurant,Breakfast Spot,Mediterranean Restaurant,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Food Court,Food,Fast Food Restaurant,Falafel Restaurant


In [26]:
# Cluster 2
pune_merged.loc[pune_merged['Cluster Labels'] == 2, pune_merged.columns[[1] + list(range(5, pune_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,18.520306,Snack Place,Bakery,Burrito Place,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant
52,18.459712,Snack Place,Bakery,Breakfast Spot,Café,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food
54,18.508672,Vegetarian / Vegan Restaurant,Snack Place,Bakery,Breakfast Spot,Café,Seafood Restaurant,Food Truck,BBQ Joint,Buffet
60,18.515579,Indian Restaurant,Bakery,Café,Breakfast Spot,Sandwich Place,Restaurant,Maharashtrian Restaurant,Seafood Restaurant,Food
63,18.511282,Café,Breakfast Spot,Comfort Food Restaurant,Food Truck,Food Court,Food,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant
69,18.510289,Indian Restaurant,Café,Vegetarian / Vegan Restaurant,Bistro,Fast Food Restaurant,Chinese Restaurant,Gastropub,Donut Shop,Asian Restaurant
72,18.520306,Snack Place,Bakery,Burrito Place,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant
75,18.520306,Snack Place,Bakery,Burrito Place,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant
76,18.520306,Snack Place,Bakery,Burrito Place,Vegetarian / Vegan Restaurant,Creperie,Food Truck,Food Court,Food,Fast Food Restaurant
81,18.519284,Indian Restaurant,Food,Seafood Restaurant,Sandwich Place,Bakery,Burrito Place,Maharashtrian Restaurant,Café,Vegetarian / Vegan Restaurant
